In [132]:
import json
import requests
from bs4 import BeautifulSoup

import os

In [133]:
def docx_html_url(_id): 
    return f'https://hudoc.echr.coe.int/app/conversion/docx/html/body?library=ECHR&id={_id}'

In [134]:
def articles_url(_number, _violation, _result_count): 
    violation = 'nonviolation'
    if(_violation): violation = violation[3:]
        
    return f'https://hudoc.echr.coe.int/app/query/results?query=contentsitename%3AECHR%20AND%20(NOT%20(doctype%3DPR%20OR%20doctype%3DHFCOMOLD%20OR%20doctype%3DHECOMOLD))%20AND%20((languageisocode%3D%22ENG%22))%20AND%20((documentcollectionid%3D%22GRANDCHAMBER%22)%20OR%20(documentcollectionid%3D%22CHAMBER%22))%20AND%20(({violation}%3D{_number}))&select=itemid,docname,&sort=&start=20&length={_result_count}&rankingModelId=11111111-0000-0000-0000-000000000000'

In [151]:
def articles_json(_number, _violation):
    results = []
    violation = 'nv'
    if(_violation): violation = violation[1:] 
        
    json_data = requests.get(articles_url(_number, _violation, 1000)).json()

        
    for result in json_data['results']:
        results.append({'id': result["columns"]["itemid"], 'name': result["columns"]["docname"], 'violation': violation})
    
    return results

In [152]:
def article_get(_id):
    url = docx_html_url(_id)
    
    r = requests.get(url).text
    soup = BeautifulSoup(r, "html5lib")
    
    for script in soup(["script", "style"]):
        script.extract()
        
    text = soup.get_text()
    
    lines = (line.strip() for line in text.splitlines())
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    text = '\n'.join(chunk for chunk in chunks if chunk)
    text = text.replace('\xa0', ' ')
    
    return text

In [153]:
def articles_generator(_number, _violation):
    violation = 'nonviolation'
    if _violation: violation = violation[3:]
        
    path = f'hudoc/Article{_number}/{violation}'
    
    os.makedirs(path, exist_ok=True)
    
    for article in articles_json(_number, _violation):
        f = open(f'{path}/{article["id"]}.txt', 'w')
        f.write(article_get(article["id"]))

In [155]:
from multiprocessing.dummy import Pool as ThreadPool 

articles = [
    ['3', True],
    ['3', False],
    ['6', True],
    ['6', False],
    ['8', True],
    ['8', False]
]
    
pool = ThreadPool(6) 
pool.starmap(articles_generator, zip(articles_num, articles_vio))

ProxyError: HTTPSConnectionPool(host='hudoc.echr.coe.int', port=443): Max retries exceeded with url: /app/conversion/docx/html/body?library=ECHR&id=001-172392 (Caused by ProxyError('Cannot connect to proxy.', OSError(0, 'Error')))